In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import numpy as np
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Masking

Using TensorFlow backend.
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\silva\AppData\Roa

In [2]:
days = 30

In [3]:
def scale(training_set):
    # Feature Scaling
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    # Creating a data structure with 'days' time-steps and 1 output
    X_train = []
    y_train = []
    for i in range(days, training_set.shape[0]):
        X_train.append(training_set_scaled[i-days:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    return X_train, y_train, sc

In [4]:
def train_LSTM(X_train, y_train):
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape = (X_train.shape[1], 1)))
    #Adding the first LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    # Adding a second LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a third LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a fourth LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the RNN to the Training set
    model.fit(X_train, y_train, epochs = 30, batch_size = 32)
    return model

In [5]:
def predict(dataset_train, dataset_test, model, sc):
    dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
    inputs = dataset_total[len(dataset_total) - dataset_test.shape[0] - days:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(days, inputs.shape[0]):
        X_test.append(inputs[i-days:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    print(X_test.shape)
    # (273, 'days', 1)
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    return X_test, predicted_stock_price

In [6]:
def plot_results(dataset_test, predicted_stock_price):
    # Visualising the results
    plt.plot(dataset_test.values, color = 'red', label = 'Real')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted')
    plt.xticks(np.arange(0,dataset_test.shape[0],50))
    plt.xlabel('Time')
    plt.ylabel('Index')
    plt.legend()
    plt.show()

In [7]:
def impute_with_prediction(df_orig, df_imputed):
    for i in df_orig.index:
        if pd.isna(df_orig.loc[i]):
            df_orig.at[i] = df_imputed.loc[i]
    return df_orig

In [8]:
INPUT = 'output/\CompleteWeeklyIndexes'
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)

In [ ]:
for i in range(0,30):
    df_train_complete = None
    df_test_complete = None
    for col in df.columns:
        test_set = df.loc['2015-01-02':, col].fillna(-1).values #Selecting test data
        training_set = df.loc['2000-01-01':'2015-01-01', col].fillna(-1).values #Selecting training data
        training_set = training_set.reshape(-1, 1)
        X_train, y_train, sc = scale(training_set)
        model = train_LSTM(X_train, y_train)
        # Dados de treinamento
        predicted_stock_price_train = model.predict(X_train)
        predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)
        #plot_results(df.loc['2000-01-01':'2015-01-01', col].fillna(-1), predicted_stock_price_train)
        # Dados de teste
        X_test, predicted_stock_price = predict(df.loc['2000-01-01':'2015-01-01', col].fillna(-1), df.loc['2015-01-02':, col].fillna(-1), model, sc)
        #plot_results(df.loc['2015-01-02':, col].fillna(-1), predicted_stock_price)

        vazio = np.empty(days)
        predicted_stock_price_train = np.append(vazio, predicted_stock_price_train)
        series_complete_train = impute_with_prediction(df.loc['2000-01-01':'2015-01-01', col], pd.DataFrame(index=df.loc['2000-01-01':'2015-01-01', col].index, data=predicted_stock_price_train))
        df_train_complete = pd.concat([df_train_complete, pd.DataFrame(index=df.loc['2000-01-01':'2015-01-01', col].index, data=series_complete_train, columns=[col])], axis=1, join='outer')
        df_test_complete = pd.concat([df_test_complete, pd.DataFrame(data=predicted_stock_price, index = df.loc['2015-01-02':, col].index, columns = [col])], axis=1, join='outer')
    df_train_complete.to_csv(r'output\/lstm\/CompleteIndexesWeeklyTrainLSTM30_n'+str(i)+'.csv', index = True)
    df_test_complete.to_csv(r'output\/lstm\/CompleteIndexesWeeklyTestLSTM30_n'+str(i)+'.csv', index = True)

Epoch 1/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0508
Epoch 2/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0139
Epoch 3/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0071
Epoch 4/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0059
Epoch 5/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0054
Epoch 6/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0051
Epoch 7/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0063
Epoch 8/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0050
Epoch 9/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0050
Epoch 10/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0050A: 1s - loss
Epoch 11/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0047
Epoch 12/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0051
Epoch 13/30


752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 11/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00A: 2s 
Epoch 12/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 13/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 14/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 15/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 16/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 17/30
752/752 [==============================] - 3s 5ms/step - loss: 0.0000e+00
Epoch 18/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 19/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 20/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0000e+00
Epoch 21/30
752/752 [==============================] - 4s 5m

752/752 [==============================] - 4s 5ms/step - loss: 0.0031
Epoch 19/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0032
Epoch 20/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0026
Epoch 21/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0028
Epoch 22/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0032
Epoch 23/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0032
Epoch 24/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0029
Epoch 25/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0029
Epoch 26/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0024
Epoch 27/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0023
Epoch 28/30
752/752 [==============================] - 3s 3ms/step - loss: 0.0026
Epoch 29/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0025
Epoch 30/30
752/752 [=======

752/752 [==============================] - 3s 4ms/step - loss: 0.0042
Epoch 27/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0041
Epoch 28/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0046A: 0s - l
Epoch 29/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0046
Epoch 30/30
752/752 [==============================] - 2s 3ms/step - loss: 0.0040
(273, 30, 1)
Epoch 1/30
752/752 [==============================] - 22s 29ms/step - loss: 0.1285
Epoch 2/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0247
Epoch 3/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0211
Epoch 4/30
752/752 [==============================] - 4s 6ms/step - loss: 0.0184
Epoch 5/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0183
Epoch 6/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0159
Epoch 7/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0160
Epoch 8/30


752/752 [==============================] - 4s 5ms/step - loss: 0.0073
Epoch 4/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0081A: 1s - l
Epoch 5/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0068
Epoch 6/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0068A: 0s - loss: 0.
Epoch 7/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0065
Epoch 8/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0070
Epoch 9/30
752/752 [==============================] - 4s 6ms/step - loss: 0.0063
Epoch 10/30
752/752 [==============================] - 4s 6ms/step - loss: 0.0064
Epoch 11/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0064
Epoch 12/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0068
Epoch 13/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0068
Epoch 14/30
752/752 [==============================] - 4s 5ms/step - loss: 0.0066
Epoch 15/

...

752/752 [==============================] - 6s 8ms/step - loss: 0.0098
Epoch 20/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0101
Epoch 21/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0099
Epoch 22/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0100
Epoch 23/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0099
Epoch 24/30
752/752 [==============================] - 6s 8ms/step - loss: 0.0099
Epoch 25/30
752/752 [==============================] - 6s 8ms/step - loss: 0.0099
Epoch 26/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0101
Epoch 27/30
752/752 [==============================] - 6s 9ms/step - loss: 0.0099
Epoch 28/30
752/752 [==============================] - 6s 9ms/step - loss: 0.0099
Epoch 29/30
752/752 [==============================] - 6s 8ms/step - loss: 0.0099
Epoch 30/30
752/752 [==============================] - 8s 11ms/step - loss: 0.0100
(273, 30, 1)
Epoch 1/30
752

752/752 [==============================] - 6s 9ms/step - loss: 0.0033
Epoch 29/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0030
Epoch 30/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0032
(273, 30, 1)
Epoch 1/30
752/752 [==============================] - 94s 125ms/step - loss: 0.0300
Epoch 2/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0086
Epoch 3/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0069
Epoch 4/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0067
Epoch 5/30
752/752 [==============================] - 6s 9ms/step - loss: 0.0069
Epoch 6/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0072
Epoch 7/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0069
Epoch 8/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0067
Epoch 9/30
752/752 [==============================] - 7s 9ms/step - loss: 0.0066
Epoch 10/30
752/7

752/752 [==============================] - 7s 10ms/step - loss: 0.0054
Epoch 6/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0049
Epoch 7/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0051
Epoch 8/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0055
Epoch 9/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0062
Epoch 10/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0045
Epoch 11/30
752/752 [==============================] - 7s 10ms/step - loss: 0.0047
Epoch 12/30
752/752 [==============================] - 8s 11ms/step - loss: 0.0045
Epoch 13/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0045
Epoch 14/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0041
Epoch 15/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0043
Epoch 16/30
752/752 [==============================] - 8s 10ms/step - loss: 0.0043
Epoch 17/30
752/752 

752/752 [==============================] - 9s 12ms/step - loss: 0.0143
Epoch 14/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0161
Epoch 15/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0148
Epoch 16/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0146
Epoch 17/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0150
Epoch 18/30
752/752 [==============================] - 9s 13ms/step - loss: 0.0152
Epoch 19/30
752/752 [==============================] - 10s 13ms/step - loss: 0.0146
Epoch 20/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0145
Epoch 21/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0144
Epoch 22/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0143
Epoch 23/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0144
Epoch 24/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0139
Epoch 25/30
752

752/752 [==============================] - 9s 12ms/step - loss: 0.0101
Epoch 22/30
752/752 [==============================] - 10s 13ms/step - loss: 0.0097
Epoch 23/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0096
Epoch 24/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0090
Epoch 25/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0089
Epoch 26/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0096
Epoch 27/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0082
Epoch 28/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0096
Epoch 29/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0098
Epoch 30/30
752/752 [==============================] - 9s 12ms/step - loss: 0.0090
(273, 30, 1)
Epoch 1/30
752/752 [==============================] - 431s 574ms/step - loss: 0.0725
Epoch 2/30
752/752 [==============================] - 10s 14ms/step - loss: 0.0281
